This week for discussion section we will discuss the problems on HW 5.

Note: On your homework and on exams, the professor asks that you use a bottom-up
approach to solve dynamic programming problems. These notes show you how you can
use the @cache decorator to easily do things top-down, but @cache uses hash
tables, which we won't learn in this class. You can implement @cache yourself
using arrays, but in practice you will still run into problems with recursion
depth.

In [1]:
from functools import cache

## Problem 1
You may choose items (numbers) from a length $n$ array $A,$ but your items must
be spaced out by at least $k.$ Design an algorithm to maximize the sum of the
items (numbers).

## Solution
#### Main idea
Let $f(i)$ denote the answer for the subarray
$$A[: i] = [A[0], \dotsc, A[i - 1]].$$
Then we have the recurrence
$$f(i) = \max(f(i - 1), A[i - 1] + f(i - k)).$$
Our algorithm uses dynammic programming to compute the value $f(n).$

#### Code


In [2]:
# bottom-up approach
def restaurants_bu(A, k):
    n = len(A)
    cache = (n + 1) * [None]
    cumulative_max = 0
    for i in range(min(k + 1, n + 1)):
        cache[i] = cumulative_max
        if i != n:
            cumulative_max = max(cumulative_max, A[i])
    for i in range(k + 1, n + 1):
        cache[i] = max(cache[i - 1], A[i - 1] + cache[i - k])
    return cache[n]

assert restaurants_bu([2, 1, 5, 6, 5, 1, 1, 3], 2) == 15
assert restaurants_bu([5, -1, 5, -1, 5], 1) == 15
assert restaurants_bu([5, -1, 5, -1, 5], 3) == 10
assert restaurants_bu([1], 1000) == 1
assert restaurants_bu(1_000 * [1], 1) == 1_000
assert restaurants_bu(1_000_000 * [1], 1) == 1_000_000
assert restaurants_bu(1_000_000 * [1], 3) == 333_334

# top-down approach (for your reference)
def restaurants_td(A, k):
    n = len(A)
    @cache
    def helper(i):
        if i <= 0:
            return 0
        else:
            return max(helper(i - 1), A[i - 1] + helper(i - k))
    return helper(n)

assert restaurants_td([2, 1, 5, 6, 5, 1, 1, 3], 2) == 15
assert restaurants_td([5, -1, 5, -1, 5], 1) == 15
assert restaurants_td([5, -1, 5, -1, 5], 3) == 10
assert restaurants_td([1], 1000) == 1
assert restaurants_td(1_000 * [1], 1) == 1_000
# the bigger test cases fail due to recursion depth
# assert restaurants_td(1_000_000 * [1], 1) == 1_000_000
# assert restaurants_td(1_000_000 * [1], 3) == 333_334


#### Proof of correctness
Let us explain why the recurrence works. When determining
$f(i),$ can either take the item $A[i - 1]$ or leave it, and our answer is
the max of the two options. If we leave it, then the best we can do is
$f(i - 1).$ If we take it, then we cannot take the items
$A[i - 2], \dotsc, A[i - k],$ so the best we can do is $A[i - 1] + f(i - k).$
Thus
$$f(i) = \max(f(i - 1), A[i - 1] + f(i - k)),$$
as claimed.

Let us discuss the base cases and the order in which we solve the subproblems.
For our base cases $i = 0, \dotsc, k,$ we can take at most one item, so our
answer is $\max(A[: i]).$ For $i > k,$ we can use our recurrence.

#### Runtime
It is $O(n)$ because `cache[i]` is computed exactly once for each $i,$ each in
$O(1)$ time.

## Problem 4
Given a list of coin values $v_1, \dotsc, v_n$ and a price $p,$ find the
smallest number of coins whose values add up to $p.$

This problem is somewhat similar to Problem 1, and you will have time to think
about and talk about it during discussion section. Here is a template for your
solution.

## Solution template

#### Main idea
TODO

#### Code

In [ ]:
# bottom-up approach
def coins_bu(values, p):
    # TODO
    pass

assert coins_bu([1, 2, 7, 20], 31) == 4
assert coins_bu([1], 10) == 10
assert coins_bu([3, 5], 14) == 4
assert coins_bu([2, 5], 1_001) == 202
assert coins_bu([1], 1_000_000) == 1_000_000
assert coins_bu([2, 5], 1_000_001) == 200_002

# top-down approach (for fun)
def coins_td(values, p):
    # TODO
    pass

assert coins_td([1, 2, 7, 20], 31) == 4
assert coins_td([1], 10) == 10
assert coins_td([3, 5], 14) == 4
assert coins_td([2, 5], 1_001) == 202
# the biggest test cases fail due to recursion depth
# assert coins_td([1], 1_000_000) == 1_000_000
# assert coins_td([2, 5], 1_000_001) == 200_002


#### Proof of correctness
TODO

#### Runtime
TODO

## Problem 2 (hard version)
Given two arrays $A$ and $B$ of numbers, design an algorithm to find a
longest common subsequence.

Note: The problem on the HW only asks for the length of a longest common
subsequence, so this is harder.

## Partial solution

Note: This solution contains the answer to the normal version of this problem.

#### Main idea
Let $f(i, j)$ denote the length of a longest common subsequence for $A[: i]$
and $B[: j].$ This satisfies the recurrence
$$f(i, j) = \begin{cases}
1 + f(i - 1, j - 1) & \text{if }A[i - 1] = B[j - 1] \\
\max(f(i - 1, j), f(i, j - 1)) & \text{otherwise}.
\end{cases}$$
We compute $f(i, j)$ for all relevant $i, j.$ To use this to determine a longest
common subsequence, note that $f(i, j) - f(i - 1, j)$ and
$f(i, j) - f(i, j - 1)$ are both $0$ or $1,$ and they are both $1$ if and only
if $A[i - 1] = B[j - 1].$ Thus by following the trail, we can construct the
latest longest common subsequence.

#### Code

In [4]:
# top-down approach (for your reference)
def lcs_td(A, B):
    @cache
    def len_lcs(i, j):
        if i == 0 or j == 0:
            return 0
        elif A[i - 1] == B[j - 1]:
            return 1 + len_lcs(i - 1, j - 1)
        else:
            return max(len_lcs(i - 1, j), len_lcs(i, j - 1))
    reversed_lcs = []
    i, j = len(A), len(B)
    while i > 0 and j > 0:
        dfdi = len_lcs(i, j) - len_lcs(i - 1, j)
        dfdj = len_lcs(i, j) - len_lcs(i, j - 1)
        if dfdi == dfdj == 1:
            reversed_lcs.append(A[i - 1])
            i -= 1
            j -= 1
        elif dfdi == 0:
            i -= 1
        else:
            j -= 1
    return list(reversed(reversed_lcs))


# test cases

def is_subsequence(S, X):
    i, j = 0, 0
    while i < len(S) and j < len(X):
        if S[i] == X[j]:
            i += 1
        j += 1
    return i == len(S)

assert is_subsequence([1, 2, 3], [0, 1, 4, 2, 8, 6, 5, 3])
assert not is_subsequence([1, 2, 3], [0, 1, 4, 8, 6, 5, 3, 2])

A = [1, 2, 3, 4, 5, 6, 7]
B = [5, 7, 2, 4, 8, 1, 6, 7, 7]
lcs = lcs_td(A, B)
assert len(lcs) == 4 and is_subsequence(lcs, A) and is_subsequence(lcs, B)

A = [1, 1, 1, 1]
B = [1, 2, 1, 2, 1, 2]
lcs = lcs_td(A, B)
assert len(lcs) == 3 and is_subsequence(lcs, A) and is_subsequence(lcs, B)

A = [0, 0, 0, 0, 9, 8, 7, 6, 5, 4, 3, 2, 1]
B = [9, 0, 8, 7, 6, 5, 4, 3, 2, 1]
lcs = lcs_td(A, B)
assert len(lcs) == 9 and is_subsequence(lcs, A) and is_subsequence(lcs, B)

A = [1]
B = [2]
lcs = lcs_td(A, B)
assert len(lcs) == 0 and is_subsequence(lcs, A) and is_subsequence(lcs, B)

A = 1_000 * [1]
B = 1_000 * [1]
lcs = lcs_td(A, B)
assert len(lcs) == 1_000 and is_subsequence(lcs, A) and is_subsequence(lcs, B)


## Problem 3
Given a large piece of clotch of dimensions $n \times m,$ you can cut the cloth
along horizontal or vertical lines, and then cut the resulting pieces, and so
on. A piece of cloth of dimensions $w \times \ell$ sells for $P[w, \ell],$ and
$P[w, \ell] = P[\ell, w]$ for all relevant $w, \ell.$ Design an algorithm to
find the most amount of money that you can make.

## Hint
Denote by $f(w, \ell)$ the most you can make from a cloth of dimensions
$w \times \ell.$ To determine a recurrence for $f(w, \ell),$ observe that we can
leave it as is and sell it for $P[w, \ell],$ or we can cut it along any of its
$w - 1$ vertical lines or any of its $\ell - 1$ horizontal lines. Thus
$f(w, \ell)$ is the max of the numbers
$$P[w, \ell], \quad f(w - i, \ell) + f(i, \ell), \quad f(w, \ell - j) + f(w, j)$$
for $i = 1, \dotsc, w - 1$ and $j = 1, \dotsc, \ell - 1.$ (Note that there are
repeats by symmetry.)

Here are some test cases.

In [ ]:
def cloth(n, m, P):
    # TODO
    pass

P = [
    [0, 0, 0, 0],
    [0, 1, 1, 1],
    [0, 1, 1, 1],
    [0, 1, 1, 1],
]
assert cloth(3, 3, P) == 9

P = [
    [0, 0, 0, 0],
    [0, 1, 1, 1],
    [0, 1, 1, 1],
    [0, 1, 1, 10],
]
assert cloth(3, 3, P) == 10

P = [
    [0, 0, 0, 0],
    [0, 1, 1, 1],
    [0, 1, 6, 1],
    [0, 1, 1, 10],
]
assert cloth(3, 3, P) == 11

P = [
    [0, 0, 0, 0],
    [0, 1, 3, 1],
    [0, 3, 6, 1],
    [0, 1, 1, 10],
]
assert cloth(3, 3, P) == 13